<a href="https://colab.research.google.com/github/Pistolll/praktika4_TOVII/blob/main/4_Fasttext.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Векторные представления слов

# Векторные представления FastText
В предыдущих представлениях слово должно быть в словаре, чтобы можно было найти для него вектор. Но что делать, если слова нет в словаре?

Зададимся вопросом, что такое слово?

В векторном представлении "слово" это элемент словаря, а значит им может быть все что угодно: смайлики, картинки, знаки пунктуации, словосочетания и т.п.

Набор букв (символов) тоже может быть элементом словаря. А давайте все тексты разобьем на последовательности символов определенной длинны (их называют n-граммы) и будем создавать вектора для этих последовательностей.

Реальное слово можно представить несколькими такими n-граммами, объединим (сложим) вектора для них и получим вектор для настоящего слова.

Никто не запрещает иметь в словаре одновременно и настоящие слова и их n-граммы.

Но это значит что для любого слова, которого даже нет в словаре, мы можем построить вектор из векторов его n-грамм. Такой подход реализован, например, в [FastText](https://amitness.com/2020/06/fasttext-embeddings/), доступна в библиотеке gensim.

![img](https://amitness.com/images/fasttext-center-word-embedding.png)

## Сжатие моделей

"Хорошие" векторные представления, поскольку обучаются на большом количестве токенов\текстов, довольно большие. Гигабайты данных. Их трудно загружать с Colab.

Мы схитрим, будем использовать сжатые векторные представления, которые может быть похуже, но не сильно, но гораздо меньше!

Подходы к сжатию ([отсюда](https://towardsdatascience.com/compressing-unsupervised-fasttext-models-eb212e9919ca)):
- выбросить большую часть векторов, а хранить только для наиболее часто встречаемых слов и n-грамм.
- хранить вектора с меньшей точностью (float16 вместо float32).
- разбить матрицу векторов на части, кластеризовать, вместо самой части хранить только экземпляр кластера и ID этого кластера.
- факторизовать матрицу векторов произведением меньших матриц, хранить их (сильно теряется точность).

![img](https://d33wubrfki0l68.cloudfront.net/82e81d0b73bce8edc6d180dc80d8f6ecf644bac2/8b910/images/product-quantization-8.jpg)


Такие подходы реализованы в библиотеке `compress-fasttext`, основана на `gensim`, и дают векторные представления в сотни раз меньшие по объему, с несильной потерей точности.  

In [3]:
# Установка совместимых версий numpy и scipy
!pip install numpy==1.24.4 scipy==1.10.1 --quiet

# Установка compress-fasttext заново
!pip install compress-fasttext --force-reinstall --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 71.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.1/34.1 MB 15.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cvxpy 1.6.5 requires scipy>=1.11.0, but you have scipy 1.10.1 which is incompatible.
jaxlib 0.5.1 requires numpy>=1.25, but you have numpy 1.24.4 which is incompatible.
jaxlib 0.5.1 requires scipy>=1.11.1, but you have scipy 1.10.1 which is incompatible.
jax 0.5.2 requires numpy>=1.25, but you have numpy 1.24.4 which is incompatible.
jax 0.5.2 requires scipy>=1.11.1, but you have scipy 1.10.1 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 1.24.4 which is incompatible.
blosc2 3.3.2 requires numpy>=1.26, but you have numpy 1.24.4 which is 

In [1]:
import compress_fasttext

# Загружаем сжатые векторные представления
wv = compress_fasttext.models.CompressedFastTextKeyedVectors.load(
    'https://github.com/avidale/compress-fasttext/releases/download/v0.0.4/cc.en.300.compressed.bin'
)

# Пример использования — получим вектор слова
print(wv['cat'][:10])  # первые 10 элементов


[ 0.06576488 -0.04598954 -0.03292161  0.28570526  0.02751245 -0.16206452
  0.11046834  0.03447995 -0.07687995  0.02738171]


In [2]:
#from gensim.models import fasttext
#from gensim.test.utils import datapath

word='polycarbon'
print(word in wv.key_to_index)  # проверяем есть ли такое слово в словаре
print(wv[word].shape)  # вектор для него

word='carbon'
print(word in wv.key_to_index)  # проверяем есть ли такое слово в словаре
print(wv[word].shape)  # вектор для него

False
(300,)
True
(300,)


In [3]:
word='polycarbonate'
print(word in wv.key_to_index)  # проверяем есть ли такое слово в словаре
print(wv[word].shape)  # вектор для него

False
(300,)


In [4]:
wv.similarity('polycarbon','polycarbonate')

0.7487792

In [5]:
# словарь
wv.key_to_index

{',': 0,
 'the': 1,
 '.': 2,
 'and': 3,
 'to': 4,
 'of': 5,
 'a': 6,
 '</s>': 7,
 'in': 8,
 'is': 9,
 ':': 10,
 'I': 11,
 'for': 12,
 'that': 13,
 ')': 14,
 '"': 15,
 '(': 16,
 'on': 17,
 'with': 18,
 'it': 19,
 'you': 20,
 'The': 21,
 'was': 22,
 'as': 23,
 'are': 24,
 'at': 25,
 '/': 26,
 '’': 27,
 'be': 28,
 'by': 29,
 "'s": 30,
 'this': 31,
 'have': 32,
 'from': 33,
 'or': 34,
 '!': 35,
 'not': 36,
 'your': 37,
 'an': 38,
 "'": 39,
 'but': 40,
 '?': 41,
 'can': 42,
 '-': 43,
 'will': 44,
 's': 45,
 'my': 46,
 'has': 47,
 'all': 48,
 'we': 49,
 'they': 50,
 'he': 51,
 'his': 52,
 'more': 53,
 'one': 54,
 'about': 55,
 'their': 56,
 "'t": 57,
 'so': 58,
 'which': 59,
 'It': 60,
 'out': 61,
 'up': 62,
 '...': 63,
 'were': 64,
 'had': 65,
 'who': 66,
 'like': 67,
 ';': 68,
 '“': 69,
 'our': 70,
 'would': 71,
 '”': 72,
 'time': 73,
 'been': 74,
 'if': 75,
 'also': 76,
 'just': 77,
 'when': 78,
 'her': 79,
 'This': 80,
 'me': 81,
 'there': 82,
 'do': 83,
 'what': 84,
 'some': 85,
 'other

In [6]:
# даже для такой ерунды есть вектор
word1='абракадабра111___)))'
print(word1 in wv.key_to_index)  # проверяем есть ли такое слово в словаре
print(wv[word1].shape)  # вектор для него

False
(300,)


In [7]:
# отличие на один символ
word2='абракадабра111___)))F'
print(word2 in wv.key_to_index)  # проверяем есть ли такое слово в словаре
print(wv[word2].shape)  # вектор для него

False
(300,)


In [8]:
# похожесть
wv.similarity(word1,word2)

1.0

In [9]:
wv.similarity('абракадабра111___)))','dfkljfdkjlcvdlksdsdposdf,m;l')

-0.041686714

In [10]:
wv.similarity('решето','решетка')

1.0

In [11]:
# похожие слова
similarities = wv.most_similar(positive=['time'])
print(similarities)

# не связанное слово
not_matching = wv.doesnt_match("human computer interface tree".split())
print(not_matching)

# похожесть слов
sim_score = wv.similarity('computer', 'human')
print(sim_score)

[('day', 0.5973712126522565), ('period', 0.590262867944538), ('moment', 0.5866017938947804), ('last', 0.5650749486972709), ('hours', 0.5359855316309537), ('periods', 0.5333457753961995), ('spend', 0.529741088077883), ('days', 0.5251064244125222), ('year', 0.520560190606691), ('when', 0.5161178628641487)]
tree
0.2725851022412919



Находит слова, наиболее близкие по значению к слову 'time' на основе векторной близости (косинусной меры).

Результат:

'day', 'period', 'moment', 'last' и т.п. — логично, это временные понятия.

Значение (например, 0.597) — это степень похожести (чем ближе к 1.0, тем сильнее семантическая близость).

In [12]:
# Схожие слова к слову "language"
similar = wv.most_similar(positive=['language'])
print("Слова, похожие на 'language':")
for word, score in similar:
    print(f"{word} ({score:.3f})")

# Самое непохожее слово в списке
odd_word = wv.doesnt_match("python java linux elephant".split())
print(f"\nНепохожее слово: {odd_word}")

# Сходство между словами
sim = wv.similarity('python', 'java')
print(f"\nСходство между 'python' и 'java': {sim:.3f}")


Слова, похожие на 'language':
languages (0.709)
Language (0.645)
English (0.609)
linguistic (0.602)
dialect (0.578)
vocabulary (0.572)
spoken (0.543)
grammar (0.537)
syntax (0.535)
Languages (0.524)

Непохожее слово: python

Сходство между 'python' и 'java': 0.137


Модель обучена таким образом, что слова, встречающиеся в похожем контексте, располагаются близко друг к другу в многомерном векторном пространстве.
Например, если в текстах часто встречаются фразы:

"learning a language",

"English is a language",

"syntax and grammar are parts of language",

то и language, English, grammar и syntax будут рядом.